In [1]:
import numpy as np
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.0001
Capacitance = 2.
Inductance = 2.
R = 1.

# solver = "ca"
# solver = "cvx_fix_discr_sdp"
solver = "cvx_sdp_relax"

In [3]:
params = {  'b': 2.668461416208648,
            'h': 6.035346326485873,
            'd': 0.7721222828714854,
            'alpha': 0.49158864756189447,
            'beta': 0.44375266588967205 }

problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

vars, sol, sp_exp = problem.solve(solver="cvx_fix_discr_sdp", verbose=True, debug=True, params=params)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Dec 06 01:55:57 PM: Your problem has 48 variables, 3 constraints, and 0 parameters.
(CVXPY) Dec 06 01:55:57 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 06 01:55:57 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 06 01:55:57 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 06 01:55:57 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 06 01:55:57 PM: Compiling problem (target solver=CLARABEL).


In [4]:
vars.keys(), params.keys()

(dict_keys(['Z', 'lambdas']), dict_keys(['b', 'h', 'd', 'alpha', 'beta']))

In [5]:
def reshape_lamb_2d(lamb):
    size = int(np.sqrt(lamb.size)) + 1
    res = np.zeros((size, size))
    count = 0
    for i in range(size):
        for j in range(size):
            if i == j: continue
            res[i, j] = lamb[count]
            count += 1
    return res


def flatten_lamb(lamb):
    # remove diagonal entries
    res = []
    for i in range(lamb.shape[0]):
        for j in range(lamb.shape[0]):
            if i == j: continue
            res += [lamb[i, j]]
    return np.array(res).reshape(-1, 1)

In [6]:
assert np.allclose(vars["lambdas"], flatten_lamb(reshape_lamb_2d(vars["lambdas"].flatten())))

In [7]:
Z = vars["Z"]
P = co.cholseky_matrix(Z, eps=1e-9)

lamb0 = reshape_lamb_2d(vars["lambdas"].flatten())

bounds = {"P": {"ub": P, "lb":P},
          "lamb0": {"ub" : lamb0, "lb" : lamb0}}
for name, val in params.items():
    bounds[name] = {"ub": val, "lb":val}

In [8]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

gp_vars, model, sp_exp = problem.solve(solver="gp", verbose=False, debug=True, bounds=bounds, ftol=1e-7)
print(model.status)

dim_G=6, dim_F=4
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
13


In [9]:
bounds = {  "Z": {"ub": Z, "lb" : Z},
            # "lamb": {"ub" : vars["lambdas"], "lb" : vars["lambdas"]}
          }

problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

cvx_solver = [cp.CLARABEL, cp.MOSEK, 'CVXOPT'][0]
bounds, sol, sp_exp = problem.solve(solver="cvx_sdp_relax", cvx_solver=cvx_solver, verbose=True, debug=True, bounds=bounds)
bounds

dim_G=6, dim_F=4
set bounds for Z
                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Dec 06 01:56:29 PM: Your problem has 5191 variables, 134 constraints, and 0 parameters.
(CVXPY) Dec 06 01:56:29 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 06 01:56:29 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 06 01:56:29 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 06 01:56:29 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 06 01:56:30 PM: Compil

SolverError: Solver 'CLARABEL' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
# fixed Z
# {'Z': 1.3976658160664561,
#  'alpha': 836.5515168748786,
#  'beta': 836.5584902659812,
#  'lamb': 104.48776609702135,
#  'P': 1.1822291723969833}

In [ ]:
assert 1 == 0

In [ ]:
L_smooth = 1.
mu = 0.001
Inductance = 5.
R = 1.

num_fi = 3
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)

In [ ]:
bounds = {  'b': {"ub": 0.000001, "lb": 0.000001},
            'h': {"ub": 0.000001, "lb": 0.000001}}

vars, sol, sp_exp = problem.solve(solver="ca", verbose=True, debug=True, bounds=bounds)

In [ ]:
params = {'b': -6.586851988799446e-12,
 'h': 2.0050490196829352e-11,
 'd': 0,
 'alpha': 0.00021484442616463077,
 'beta': 0.18371375512916682}


params = {  'b': 0.000001,
            'h': 0.000001,
            'd': 0.,
            'alpha': 0.5,
            'beta': 0.5}

bounds = {  'b': {"ub": 0.000001, "lb": 0.000001},
            'h': {"ub": 0.000001, "lb": 0.000001}}

In [ ]:
# vars, prob, sp_exp = problem.solve(solver=solver, verbose=True, debug=True)
vars, prob, sp_exp = problem.solve(solver=solver, verbose=False, debug=True, params=params)
print(prob.status)

In [ ]:
init_vals = {}
Fi_size = vars["lambda"].size // num_fi
size_I = int(np.ceil(np.sqrt(Fi_size)))
for f_idx in range(num_fi):
    lamb_i = np.zeros((size_I, size_I))
    vec = np.maximum(vars["lambda"][f_idx * Fi_size : (f_idx+1) * Fi_size], 0)
    count  = 0
    for i in range(size_I):
        for j in range(size_I):
            if i == j: continue
            lamb_i[i, j] = vec[count]
            count += 1
    assert count == size_I * (size_I - 1)
    init_vals["lamb%d"%f_idx] = lamb_i

Z = vars["Z"]
Lamb, V = np.linalg.eigh(Z)
assert np.allclose(V @ np.diag(Lamb) @ V.T, Z)
Z_plus = V @ np.diag(np.maximum(Lamb, 0)) @ V.T
P = np.linalg.cholesky(Z_plus)

init_vals["P_full"] = P

In [ ]:
# problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)

In [ ]:
problem = co.admm_consensus(num_fi, mu, L_smooth, R, Inductance)

In [ ]:
vars, sol, sp_exp = problem.solve(solver="casadi_short", verbose=True, debug=True, bounds=bounds)#init_vals=init_vals)

In [ ]:
print(problem.opti.stats()["iter_count"])

In [ ]:
vars